In [6]:
import cv2
import numpy as np
import joblib
from skimage.feature import hog

In [7]:
svm_model = joblib.load("gesture_svm_model.pkl")
label_encoder = joblib.load("gesture_label_encoder.pkl")

In [8]:
def extract_features(image):
    return hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9)

In [11]:
cap = cv2.VideoCapture(0)  

while True:
    ret, frame = cap.read() 
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    h, w = gray.shape
    hand_region = gray[h//4: 3*h//4, w//4: 3*w//4]  

    hand_resized = cv2.resize(hand_region, (100, 100))

    features = extract_features(hand_resized)

    predicted_class = svm_model.predict([features])
    gesture_name = label_encoder.inverse_transform(predicted_class)[0]

    cv2.putText(frame, f"Gesture: {gesture_name}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.rectangle(frame, (w//4, h//4), (3*w//4, 3*h//4), (0, 255, 0), 2)
    cv2.imshow("Hand Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()